In [1]:
# Outside imports
import os
import importlib
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [86]:
import data_util.preprocess 
importlib.reload(data_util.preprocess)
from data_util.preprocess import *



import model
import train
import evaluate
import train_util
import data_util.data
import data_util.batcher
import data_util.config

importlib.reload(data_util.batcher)
importlib.reload(model)
importlib.reload(train)
importlib.reload(evaluate)
importlib.reload(train_util)
importlib.reload(data_util.config)
importlib.reload(data_util.data)

from data_util.batcher import *
from evaluate import *
from model import *
from train import *
from train_util import *
from data_util.data import *



In [3]:
# load real data
data_path = '/Users/rowancassius/Desktop/capstone/data/context_task_data.tsv'
dat = pd.read_csv(data_path, sep='\t')

In [4]:
!pwd

/Users/rowancassius/Desktop/capstone/LSTM_Summarizer


In [5]:
# load real data
data_path = '/Users/rowancassius/Desktop/capstone/data/context_task_data.tsv'
dat = pd.read_csv(data_path, sep='\t')

# fill nas
dat.fillna('', inplace=True)

# train/test split
np.random.seed(111)
dat = dat.sample(frac=1)
train_size = int(.8*dat.shape[0])
train_data = dat[:train_size]
test_data = dat[train_size:]


In [6]:
pd.set_option('display.max_colwidth', -1)

In [174]:
train_data.sample(10)

,#,Task,Context,TaskSentence,Summary,Labeler,NoRequestInContext,Urgent,NotRequest,Unsure/Discuss,RandomNumber
161,162,Please use the following two files as your detail support for finanical,,Please use the following two files as your detail support for finanical liquidations:,Use files as detail support,Rowan,,,,,0.714333
843,5159,"process from a RAC perspective, so going forward, would you please copy her",,"Cheryl - you're doing a great job of trying to keep us in the loop - Mary's going to be working with you on the administration of this brokerage account process from a RAC perspective, so going forward, would you please copy her on all correspondence you'd normally include me in?",Copy Mary on all correspondence with SENDER,Natalie,1,0,0,,0.981606
116,117,Please review and provide your thoughts.,"Brian,. Attached is a very rough draft of a 20/20 Term Sheet that is contemplated to be attached to a Letter of Intent.",Please review and provide your thoughts.,Review draft of 20/20 Term Sheet,Rowan,1,,,,0.151844
1207,5523,Please include the room and time on,"Kim, please send Victor and myself an amtel message at 11:00am AND at 1:30pm on Feb 1 to attend the meeting below.",Please include the room and time on the amtel.,Include room and time on amtel,Natalie,0,0,1,,0.369536
886,5202,so could you please prepare an ISDA Agreement for their review.,,"David, We want to be able to start performing financial trades with Frontera so could you please prepare an ISDA Agreement for their review.",Prepare ISDA Agreement for Frontera's review,Natalie,1,0,0,,0.329744
381,382,Can you give me some background on Bogdan?,Vince:,Can you give me some background on Bogdan?,Provide background on Bogan,Rowan,1,0,,,0.894518
893,5209,"Please call me at 707-9641-155 ,","Hi Jeff: I know a Jeff Dasovich, exact spelling, an attorney in Oklahoma City...I'm looking forward to talking to you about the potential for vineyard.","Please call me at 707-9641-155 , I zoom around pretty fast but should be able to get down there soon.",Call SENDER at 707-9641-155 about vineyard,Natalie,1,0,0,,0.224640
342,343,"But, I'm still here; give me a call when you get a",Thanks for the note and congratulations.. Sorry we didn't get a chance to talk before you left.,"But, I'm still here; give me a call when you get a chance - 1586.",Call SENDER,Rowan,1,0,,,0.753786
1125,5441,Please review the language under B of Contract Price of the Transaction Agreement to see if it isn't sufficient to cover the concept under item 8 of Pasadena Usage under your term sheet.,,Please review the language under B of Contract Price of the Transaction Agreement to see if it isn't sufficient to cover the concept under item 8 of Pasadena Usage under your term sheet.,Review language of Transaction Agreement,Natalie,1,0,0,,0.991959
1141,5457,No biggie ... give me a call back when you get a chance to chat some more,,No biggie ... give me a call back when you get a chance to chat some more :,Call SENDER,Natalie,1,0,0,,0.492506


##  Vocabulary Curation

In [8]:
vocab_sents = [vocab_process_text(s) for s in 
               train_data.Context.values + train_data.TaskSentence.values]

In [9]:
# fit tokenizer
tokenizer = Tokenizer(lower=False, filters='')
tokenizer.fit_on_texts(vocab_sents + list(ENT_TAGS))
assert all([t in tokenizer.word_index for t in ENT_TAGS])

In [10]:
words2vocabfile(tokenizer.word_index.keys(), os.path.join(config.log_root, 'data/vocab.txt'))

In [11]:
vocab = Vocab.from_vocab_file(os.path.join(config.log_root, 'data/vocab.txt'))

Finished constructing vocabulary of 4178 total words. Last word added: a


In [175]:
i= 116
context = train_data.loc[i]['Context']
task    = train_data.loc[i]['TaskSentence']
summary = train_data.loc[i]['Summary']


context = article_process_text(context)
task = article_process_text(task)
summary = summary_process_text(summary)

In [176]:
summary

'review draft of 20/20 Term Sheet'

In [177]:
import data_util.batcher
importlib.reload(data_util.batcher)
from data_util.batcher import *

In [178]:
ex = TaskExample(context, task, summary, vocab)

In [179]:
len(ex.enc_input)

27

In [180]:
ex.dec_input

[2, 519, 893, 986, 3207, 0]

In [181]:
ex.entity_label_map

{'Brian, Attached': 'ORG', '20/20': 'CARDINAL', 'a Letter of Intent': 'ORG'}

In [182]:
ex.article_oovs

['Brian, Attached', '20/20', 'a Letter of Intent']

In [183]:
ex.summary_tokens

['review', 'draft', 'of', '20/20', 'Term Sheet']

In [184]:
ex.sum_ids_extend_vocab

[519, 893, 986, 4179, 0]

In [185]:
ex.dec_input

[2, 519, 893, 986, 3207, 0]

In [138]:
ex.summary.ents

()

In [191]:
doc = nlp("Hey George, please email me asap at rowan@gmail.net. Take the survey at extraction.com. Also my friend at Google wants you to edit Article III, and call him at (445) 7680099")

In [192]:
{e.text: e.label_ for e in doc.ents}

{'George': 'PERSON',
 'rowan@gmail.net': 'EMAILADDRESS',
 'extraction.com': 'WEBSITE',
 'Google': 'ORG',
 'Article III': 'PRODUCT',
 '(445) 7680099': 'PHONENUMBER'}

In [ ]:
''

In [193]:
list(doc)

[Hey,
 George,
 ,,
 please,
 email,
 me,
 asap,
 at,
 rowan@gmail.net,
 .,
 Take,
 the,
 survey,
 at,
 extraction.com,
 .,
 Also,
 my,
 friend,
 at,
 Google,
 wants,
 you,
 to,
 edit,
 Article III,
 ,,
 and,
 call,
 him,
 at,
 (445) 7680099]